# 第一部分：使用指令数据对基底模型进行有监督微调
在本作业的第一部分，我们将使用Qwen2.5-0.5B基底模型以及alpaca指令数据集，体验如何对LLM做指令微调的训练。

> 关于Transformer的基本使用教程，可以参考官方推出的[LLM Course](https://huggingface.co/learn/llm-course/chapter2/3)。本次作业要求同学们手写训练代码，不能使用里面提供的Trainer API，关于如何使用PyTorch训练模型，可以参照[这个教程](https://huggingface.co/docs/transformers/v4.49.0/en/training#train-in-native-pytorch)。

> 对于使用Kaggle进行作业的同学，这里有一份[Kaggle基础使用](https://www.kaggle.com/code/cnlnpjhsy/kaggle-transformers)的简单教学供参考。

In [ ]:
# 如果缺失必要的库，可以使用下面的命令安装
!pip install torch transformers datasets

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import datasets

## 加载模型、tokenizer与数据集
本次作业，我们使用通义千问的Qwen2.5-0.5B预训练模型进行微调。对于在本地部署的同学，请事先将模型文件下载到本地；对于在kaggle上进行作业的同学，可以依照kaggle上的教程，将`MODEL_PATH`与`DATASET_PATH`修改为Input中的路径。

In [ ]:
MODEL_PATH = "Qwen2.5-0.5B"
DATASET_PATH = "alpaca/alpaca_data_cleaned.json"

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, device_map="auto", torch_dtype="auto")
print(model)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

dataset = datasets.Dataset.from_json(DATASET_PATH)
for sample in dataset.select(range(10)):    # 查看前10个样本。思考应该怎么将样本组织成单条完整文本？
    print(sample)

Qwen为基底模型也提供了对话模板（chat template），对话模板中含有一些特殊的token，可以帮助我们区分说话人的轮次（思考一下为什么要区分？）。我们可以直接以下述“轮次对话”的方式，构造一个样例文本。

In [ ]:
tokenizer.apply_chat_template([
    {"role": "user", "content": "This is a question."},
    {"role": "assistant", "content": "I'm the answer!"}
], tokenize=False
)

可以看到每一轮次的对话都以`<|im_end|>`这个token结束。但是基底模型是没有在对话上经过优化的，它并不认得这个终止符。因此我们需要修改tokenizer的终止符，使其知道什么token代表一个对话轮次的结束。

In [ ]:
print(tokenizer.eos_token)  # 原来的终止符
tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids("<|im_end|>")
tokenizer.pad_token_id = tokenizer.eos_token_id
model.generation_config.eos_token_id = tokenizer.eos_token_id  # 也要修改模型的终止符

为了与训练后的模型做对比，我们先使用模型自带的generate方法测试一下这个基底模型会生成什么样的文本：

In [ ]:
messages = [
    {"role": "user", "content": "Give me a brief introduction to Shanghai Jiao Tong University."},
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
with torch.no_grad():
    lm_inputs_src = tokenizer([text], add_special_tokens=False, return_tensors="pt").to(model.device)
    generate_ids = model.generate(**lm_inputs_src, max_new_tokens=150, do_sample=False)
pred_str = tokenizer.decode(generate_ids[0][lm_inputs_src.input_ids.size(1):], skip_special_tokens=True)
print(pred_str)

## 处理数据集
原始的alpaca数据集是纯文本形式，而非模型能够接受的token。我们需要先将这些文本tokenize，再传给模型。

在指令微调阶段，我们常常希望模型只在模型要生成回答的部分上做优化，而不在问题文本上做训练，这需要我们特别设计传入的标签。请完成下述的`tokenize_function`函数，将数据集的指令样本tokenize，并传回输入模型的`input_ids`以及用于<b>仅在output部分计算损失</b>的标签`labels`。

In [ ]:
import copy
def tokenize_function(sample):
    input_ids = None
    labels = None
    # TODO: 完成函数，使之能够对数据集中的每个样本进行正确的tokenize，生成训练时用于输入模型的input_ids和labels。
    # 思考一下，labels的标签应该如何设置，才能让模型只对output的部分进行计算loss？
    pass

    return {"input_ids": input_ids, "labels": labels}

tokenized_dataset = dataset.map(
    tokenize_function, remove_columns=dataset.column_names
).filter(
    lambda x: len(x["input_ids"]) <= 512
)

定义一个DataLoader，用于从中获取模型能够处理的tokenized输入。  
> <b>【附加1】（3分）</b>通过从dataloader中成批取出数据，可以提升计算效率。你能够设计`collate_fn`，使之能以`batch_size > 1`的方式获取数据吗？

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    input_ids = None
    attention_mask = None
    labels = None
    # TODO: 完成函数，使之能够对取出的batch样本进行处理，生成适合模型输入的input_ids, attention_mask和labels
    pass

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

train_dataloader = DataLoader(tokenized_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

## 训练模型
准备好tokenized后的数据后，就可以对模型进行训练了。请手动编写用于训练的循环，计算损失并反传。

在向model传入labels时，Transformer模型内部会自动计算损失；但为了让同学们理解损失的内部计算机制，我们要求**不向模型forward中传入labels，而是手动将模型的最终输出logits与labels相比对，并计算损失。**  
> <b>【附加1】</b>从dataloader中成批获取数据后，要将整个batch一次性输入到模型中（并非是使用循环逐个处理批次输入），获取所有样例的loss，并正确计算损失。

In [ ]:
from tqdm.notebook import tqdm

step = 0
# TODO: 定义你的优化器与损失函数
optimizer = None
loss_fn = None

model.train()
for epoch in range(3):
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
        loss = None
        # TODO: 手动完成单步的训练步骤
        pass

        step += 1
        if step % 100 == 0:
            print(f"Step {step}\t| Loss: {loss.item()}")
    model.save_pretrained(f"output/checkpoint-epoch-{epoch + 1}")
    tokenizer.save_pretrained(f"output/checkpoint-epoch-{epoch + 1}")

测试训练后的模型效果。如果训练正常，模型应当能回答出通顺的语句，并在回答结束后自然地停止生成。

In [ ]:
sft_model = AutoModelForCausalLM.from_pretrained("output/checkpoint-epoch-3", device_map="auto", torch_dtype="auto")
messages = [
    {"role": "user", "content": "Give me a brief introduction to Shanghai Jiao Tong University."},
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
with torch.no_grad():
    lm_inputs_src = tokenizer([text], add_special_tokens=False, return_tensors="pt").to(sft_model.device)
    generate_ids = sft_model.generate(**lm_inputs_src, max_new_tokens=150, do_sample=False)
pred_str = tokenizer.decode(generate_ids[0][lm_inputs_src.input_ids.size(1):], skip_special_tokens=True)
print(pred_str)

如果模型行为正常，就可以继续前往大作业的第二部分了！

# 第二部分：使用LLM做推理生成，并解码为自然文本
在这一部分，我们将体验LLM是如何逐token进行生成、并解码出自然文本的。我们需要手动实现一个`generate`函数，它能够直接接受用户的自然文本作为输入，并同样以自然文本回复。

In [ ]:
MODEL_PATH = "output/checkpoint-epoch-3"    # 你训练好的模型路径

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, device_map="auto", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids("<|im_end|>")
tokenizer.pad_token_id = tokenizer.eos_token_id
model.generation_config.eos_token_id = tokenizer.eos_token_id

## 实现generate
请实现下述的generate函数，手动进行模型推理、生成与解码。

这个generate函数至少能够接受一个字符串`query`作为输入，限制最大生成token数`max_new_tokens`，并用`do_sample`选择是采用采样还是贪婪搜索进行生成。在使用采样策略生成时，允许设置基础的采样生成参数`temperature`、`top_p`和`top_k`。关于不同的生成策略是如何工作的，可以学习这篇[博客](https://huggingface.co/blog/how-to-generate)。  
**禁止使用模型自带的`model.generate`方法！**

> <b>附加2（3分）</b>你能够利用模型的批次输入特性（并非是使用循环逐个处理批次输入），成批次地输入文本、并同时生成新token吗？此时`query`应该可以接受一个字符串列表作为输入。

> <b>附加3（3分）</b>束搜索（Beam search）允许在解码过程中保留数个次优序列，通过生成过程中维护这些序列，模型能够生成整体更为合理的句子，改善了贪婪搜索中可能会陷入局部最优的问题。你可以在已有的贪婪搜索与采样两种生成策略的基础上实现束搜索吗？此时`num_beams`应允许大于1的值。  
关于束搜索，这里有一个[可视化Demo](https://huggingface.co/spaces/m-ric/beam_search_visualizer)演示其运作机理。

In [ ]:
from typing import Union, List

def generate(
    model: AutoModelForCausalLM,
    query: Union[str, List[str]],
    max_new_tokens: int = 1024,
    do_sample: bool = False,
    temperature: float = 1.0,
    top_p: float = 0.9,
    top_k: int = 50,
    num_beams: int = 1,
    length_penalty: float = 1.0,
) -> Union[str, List[str]]:
    """
    使用模型model进行文本生成。
    Args:
        model: 用于生成的语言模型
        query: 用户输入的查询。可以是单个字符串，或者是一个字符串列表【附加2】
        max_new_tokens: 生成的最大新token数量
        do_sample: 是否使用采样生成文本。仅当为True时，后续的temperature、top_p、top_k参数才会生效
        temperature: 采样时的温度参数
        top_p: 采样时的top-p参数
        top_k: 采样时的top-k参数
        num_beams: 束搜索同时维护的束的数量。仅当`num_beams > 1`时，才会启用束搜索【附加3】
        length_penalty: 启用束搜索时的长度惩罚系数【附加3】
    Returns:
        生成的文本。如果输入是单个字符串，则返回单个字符串；如果输入是字符串列表，则返回字符串列表【附加2】
    """
    # TODO: 完成函数，实现文本生成
    pass

## 测试generate的效果
请同学们运行下述单元格，测试你的实现。除了下面提到的句子，同学们也可以自定义更多情况下的输入文本，探究模型在面对不同输入时采用不同解码策略的表现。

In [ ]:
print("#1 贪心解码")
query1 = ["Give me a brief introduction to Shanghai Jiao Tong University.", "介绍一下上海交通大学。", "What is the capital of China?"]
# 如果没有实现附加2，请用循环的方式依次解码query1里的每个字符串并打印出来
for i, response in enumerate(generate(model, query1, max_new_tokens=256, do_sample=False)):
    print(f"[{i}] 问：{query1[i]}\n答：{response}")

print("\n#2 采样解码")
query2 = "Tell me a joke about computers."
for i in range(5):
    response = generate(model, query2, do_sample=True, temperature=0.7, top_p=0.9, top_k=50)    # 可以试试调整这些采样超参数
    print(f"[{i}] 问：{query2}\n答：{response}")

print("\n#3 【附加3】束搜索解码")
query3 = "What is the sum of the first 100 natural numbers? Please think step by step."
response = generate(model, query3, num_beams=4, length_penalty=1.0)
print(f"问：{query3}\n答：{response}")